# SQL Injection

### The purpose of this exercise is to demonstrate how unsanitized database queries can lead to leaks of personal information (e.g., in a web based application where people enter their names and other information). 

### In this exercise we will:

<ul>
<li>Connect to a database.</li>
<li>Submit a legitimate query.</li>
<li>Submit a variety of illicit queries.</li>
<li>Demonstrate how to sanitize database inputs to reduce the occurance of SQL injection attacks.</li>
</ul>

A sample SQLite relational database has been created that stores information about a set of employees, including their names, social security numbers, home address, and other information.  

A common way to interact with relational databases is via submission of **queries**.  _SQL_ is one agreed upon way to structure these queries.

<blockquote>SQL is short for the <b>Structured Query Language</b>.</blockquote>

The follow code snippet demonstrates how to connect to the database and query that information using the SQL _SELECT_ statement.

In [ ]:
from background import *

connection = create_connection("test.db")

select_users = "SELECT * from EMPLOYEE"
users = execute_read_query(connection, select_users)

for user in users:
    print(user) 

There are situations where one may not wish a user to be able to see information about arbitrary entries in the database.  For instance, the next code snippet requires a user to enter their SSN into a webform in order to see their current information.  The assumption here is that one only knows their own SSN, and not the SSN of their co-workers.  

**Exercise:** Try the example for the employee whose SSN is 123456789. (make sure to press enter to confirm the value)

In [ ]:
ssn=input("Enter your SSN:")
select_users = "SELECT * from employee where ssn=" + ssn

users = execute_read_query(connection, select_users)

for user in users:
    print(user) 

Very good.  Now a user can only see information for SSN numbers they know.  There is a first, obvious, vulerability in that a user could guess random SSNs and, perhaps, leak information about their co-workers, but the search space is large.  A second, more insidious vulnerability is known as _SQL Injection_ where additional SQL code can be fed into the application to change the results.  

**Exercise:** When running the following box, append "OR TRUE" to the previous SSN to see what happens.

In [ ]:
ssn=input("Enter your SSN:")
select_users = "SELECT * from employee where ssn=" + ssn

users = execute_read_query(connection, select_users)

for user in users:
    print(user) 

As one can see, because of the mechanism the code uses to build its query, it is vulnerable to manipulation of the _WHERE_ clause.  

<blockquote>Actions taken to reduce the likelihood of query manipulation are known as query <b>sanitation</b></blockquote>

<img src="https://imgs.xkcd.com/comics/exploits_of_a_mom.png">

<center>Figure from https://xkcd.com/327/.</center>

Let's run a new version that sanitizes the user input to ensure it only contains a valid SSN.

**Exercise:** Try running the following block with and without the "OR TRUE" and you'll see only the valid SSN works.

In [ ]:
ssn=input("Enter your SSN:")
if(isSSN(ssn)):
    select_users = "SELECT * from employee where ssn=" + ssn

    users = execute_read_query(connection, select_users)

    for user in users:
        print(user) 

Let's try another example.  Let's say an employee has the following script to add new employees to the database.  Can you spot the vulnerability?

In [ ]:
first_name = "Abraham"
last_name = "Lincoln"
ssn = "222222222"
department_number = "1";

add_user = """
INSERT INTO EMPLOYEE (Fname,Lname,Ssn,Dno)
VALUES
(\"""" + first_name + "\",\"" + last_name + "\"," + ssn + "," + department_number + ");"

execute_queries(connection,add_user)

At first, it might seem like there is no good way to alter the query to expose extra information, because it is constrainted by the values in the query's _INSERT_ clause.  However, if you notice, the author of the script allows multiple queries to be run at once.  This is good for initializing a database, but not a great idea for regular use.

This allows one to use another class of exploits that involves appending additional queries and running them all at the same time.  

**Exercise:** Run the following code sample and use the input (you can copy and paste the whole line):

1); UPDATE EMPLOYEE SET Salary=100000 WHERE Lname="English"; DELETE FROM EMPLOYEE WHERE (Lname="Washington"

Then see what the fully resolved query looks like when printed.

In [ ]:
first_name = "George"
last_name = "Washington"
ssn = "555555555"
department_number = input()

add_user = """
INSERT INTO EMPLOYEE (Fname,Lname,Ssn,Dno)
VALUES
(\"""" + first_name + "\",\"" + last_name + "\"," + ssn + "," + department_number + ");"

print(add_user)

One can see that with some clever formating we have spliced two additional queries into a single SQL compliant string that:

- adds a dummy employee
- adjusts an employee's salary
- removes the dummy employee

**Exercise:** Run the following code to use the exploit query and print the entire table.  Compare the table to our original printout and see what changed.

In [ ]:
execute_queries(connection,add_user)

select_users = "SELECT * from employee"
users = execute_read_query(connection, select_users)

for user in users:
    print(user) 

If one compares this to the original table, one will see that Joyce has received a nice raise.  While it is convient to allow execution of batches to queries to, for instance, initialize your database, you should restrict execution to one query at a time in public facing scripts.  Similarly, this shows another aspect of _trust_, if one is giving another write access to a database, then ensure they are of sufficient character not to abuse these privledges.

In summary, if you construct a dynamic SQL query based on user input make sure to sanitize your inputs!

# Advanced ideas:

A problem with this database is that database is that SSNs are stored in plaintext.  One way to solve this problem, would be to store a user password via the techniques described earlier in module and using that password to encrypt sensitive information.